In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
!unzip -q "/content/drive/MyDrive/brain_tumor_dataset.zip" -d "/content/brain_tumor_dataset"


In [4]:
import os, shutil
import pandas as pd
from sklearn.model_selection import train_test_split

raw_path = "/content/brain_tumor_dataset"
output_path = "/content/brain_mri_split"
classes = ['yes', 'no']

data = []
for cls in classes:
    cls_dir = os.path.join(raw_path, cls)
    for fname in os.listdir(cls_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            data.append({"path": os.path.join(cls_dir, fname), "label": cls})

df = pd.DataFrame(data)

train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)

splits = {"train": train_df, "val": val_df, "test": test_df}

for split, split_df in splits.items():
    for cls in classes:
        split_cls_dir = os.path.join(output_path, split, cls)
        os.makedirs(split_cls_dir, exist_ok=True)
        for _, row in split_df[split_df["label"] == cls].iterrows():
            shutil.copy(row["path"], os.path.join(split_cls_dir, os.path.basename(row["path"])))


In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

data_dir = "/content/brain_mri_split"

train_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_ds = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_ds = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=0)


In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

class_labels = np.unique(train_ds.targets)
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=class_labels,
                                     y=np.array(train_ds.targets))

weights = torch.tensor(class_weights, dtype=torch.float).to(device)


In [8]:
import timm
import torch.nn as nn

model_deit = timm.create_model(
    'deit_base_patch16_224',
    pretrained=True,
    num_classes=2
)
model_deit.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model_deit.parameters(), lr=3e-4)


In [10]:
from tqdm import tqdm

for epoch in range(10):
    model_deit.train()
    total_loss = 0
    for imgs, labels in tqdm(train_loader, desc=f"DeiT Epoch {epoch+1}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_deit(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


DeiT Epoch 1: 100%|██████████| 23/23 [05:21<00:00, 13.98s/it]


Epoch 1 Loss: 17.7392


DeiT Epoch 2: 100%|██████████| 23/23 [05:21<00:00, 13.98s/it]


Epoch 2 Loss: 13.9511


DeiT Epoch 3: 100%|██████████| 23/23 [05:21<00:00, 13.99s/it]


...

Epoch 8 Loss: 14.4858


DeiT Epoch 9: 100%|██████████| 23/23 [05:21<00:00, 13.96s/it]


Epoch 9 Loss: 12.4161


DeiT Epoch 10: 100%|██████████| 23/23 [05:21<00:00, 13.97s/it]

Epoch 10 Loss: 7.7439


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import torch

model_deit.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_deit(imgs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


In [12]:
# Accuracy
acc = accuracy_score(y_true, y_pred)

# Precision
prec = precision_score(y_true, y_pred, zero_division=0)

# Recall
rec = recall_score(y_true, y_pred, zero_division=0)

# F1-score
f1 = f1_score(y_true, y_pred, zero_division=0)

# Classification Report
report = classification_report(y_true, y_pred)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

print("📋 Classification Report:\n", report)
print("🎯 Accuracy:", acc)
print("🎯 Precision:", prec)
print("🎯 Recall:", rec)
print("🎯 F1 Score:", f1)
print("🌀 Confusion Matrix:\n", cm)


📋 Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.67      0.80        15
           1       0.82      1.00      0.90        23

    accuracy                           0.87        38
   macro avg       0.91      0.83      0.85        38
weighted avg       0.89      0.87      0.86        38

🎯 Accuracy: 0.868421052631579
🎯 Precision: 0.8214285714285714
🎯 Recall: 1.0
🎯 F1 Score: 0.9019607843137255
🌀 Confusion Matrix:
 [[10  5]
 [ 0 23]]


In [13]:
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

y_probs = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model_deit(imgs)
        probs = F.softmax(outputs, dim=1)
        y_probs.extend(probs[:, 1].cpu().numpy())

roc_auc = roc_auc_score(y_true, y_probs)

print("🏆 ROC-AUC Score:", roc_auc)


🏆 ROC-AUC Score: 0.9246376811594204
